In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/offending/aita_list_comment_authors.csv
/kaggle/input/reladvice/reladviceposts (1).csv


In [2]:
# Install all necessary components

!pip install praw


     |████████████████████████████████| 143 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 36.2 MB/s eta 0:00:01


In [3]:
# Import libraries
from bs4 import BeautifulSoup as soup
import pandas as pd

import requests
import json
import urllib
import praw

import logging
import prawcore
logger = logging.getLogger(__name__)

import time

from multiprocessing import Process, Manager, Pool
from datetime import datetime

import argparse
import requests
import json
import re

import random
import numpy as np



reddit = praw.Reddit(client_id='QDrWU3_ioojm2A',
                     client_secret='pds-1UaJb6NQ3kRtvXPPdheT3k0',
                     password='reddit12344',
                     user_agent='cse6240project',
                     username='cse6240project')



In [ ]:
# Hyperparameters
HTTPS = "https://api.pushshift.io/reddit/submission/search/?after="

##############################################
#r/AskReddit Jan 25, 2008                    #
#r/relationships Created Jul 9, 2008         #
#r/AmITheAsshole Created Jun 8, 2013         #
#r/relationship_advice Created Jun 14, 2009  #
#r/OffMyChest Created Created Oct 23, 2009   #
##############################################
SUBREDDIT = ["AskReddit", "relationships", "AmITheAsshole", "relationship_advice", "OffMyChest"]

BEFORE = [1201219200, 1215561600, 1370649600, 1244937600, 1256256000]

ONEYEAREPOCH =  31556926

SIZE = 1000

In [ ]:
# df = None

# for i in range(0, 11):
#     #link = "https://api.pushshift.io/reddit/submission/search/?after=1506816000&before=1506902400&sort_type=score&sort=desc&subreddit=relationship_advice&size=1"
    
#     link = HTTPS + str(BEFORE[3]+ONEYEAREPOCH*i) + "&before=" + str(BEFORE[3]+(i+1)*ONEYEAREPOCH) + "&sort_type=score&sort=desc&subreddit=" + SUBREDDIT[3] + "&size=" + str(SIZE)
#     print(link)
#     try:
#         data = json.load(urllib.request.urlopen(link))
#         if df is not None:
#             df = df.append(pd.DataFrame.from_dict(data['data']), ignore_index=True)
#         else:
#             df = pd.DataFrame.from_dict(data['data'])
#         time.sleep(5)
#     except Exception:
#         time.sleep(15)
#         print("sleeping")

In [ ]:
df = pd.read_csv("/kaggle/input/reladviceposts (1).csv")

In [ ]:
len(df)

In [ ]:
df = df[5000:7000]

In [ ]:
len(df)

In [ ]:
def parallelize_comments(df):
    vfunc = np.vectorize(comment_map)
    df['commentmap'] = vfunc(df['url'])
    return df

In [ ]:
def comment_map(posturl):
    submission = reddit.submission(url = posturl)
    #print(posturl)
    try:
        submission.comments.replace_more(limit= 0)
    except Exception:
        print("sleeping")
        time.sleep(5)
    result = submission.comments.list()
    #print(result)
    return result

In [ ]:
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
    
train = parallelize_dataframe(df, parallelize_comments)

In [ ]:
train

In [ ]:
train.to_csv('reladvice_posts1.csv',index=False) 

In [4]:
train = pd.read_csv("/kaggle/input/offending/aita_list_comment_authors.csv")  


In [6]:
print(np.where(train.applymap(type)!=str)) #988947
print(np.where(train.applymap(type)==float)) #988947

(array([     0,      0,      1, ..., 225438, 225439, 225439]), array([1, 2, 1, ..., 2, 1, 2]))
(array([], dtype=int64), array([], dtype=int64))


In [7]:
#train['comment_map'][11805]
print(len(train))

225440


In [ ]:
def parallelizecomments(comment_df):
    try:
        comment_df['comment_body'] = [elem.body for elem in comment_df['comment_map']]
        comment_df['comment_author'] = [elem.author for elem in comment_df['comment_map']]
    except AttributeError as error:
        #print(elem)
        print("AttributeError")
    except Exception as error:
        print("Exception")
    return comment_df

In [ ]:
comment_df = pd.DataFrame()
comment_df['url'] = train['url']
comment_df['comment_map'] = train['commentmap']

In [ ]:

comment_df = comment_df.explode('comment_map')
comment_df.reset_index(drop=True, inplace=True)
comment_df.dropna(inplace=True)



In [ ]:
comment_df = parallelize_dataframe(comment_df, parallelizecomments)

In [ ]:
#comment_df

In [ ]:
# try:
#     comment_df = pd.DataFrame()
#     comment_df['url'] = train['url']
#     comment_df['comment_map'] = train['commentmap']
#     comment_df = comment_df.explode('comment_map')
#     comment_df.reset_index(drop=True, inplace=True)
#     comment_df.dropna(inplace=True)
#     comment_df['comment_body'] = [elem.body for elem in comment_df['comment_map']]
#     comment_df['comment_author'] = [elem.author for elem in comment_df['comment_map']]
# except AttributeError as error:
#     #print(elem)  
#     print("AttributeError")
# except Exception as error:
#     print("Exception")

In [ ]:
comment_df.to_csv('reladvice_comments1.csv',index=False)   

In [ ]:
# comment = reddit.comment('f2gtri5')
# print(comment.body)

In [9]:
author_df = pd.read_csv("/kaggle/input/offending/aita_list_comment_authors.csv") 


In [10]:
author_df

,comment_author,Hurtful,Freq hurtful words
0,AutoModerator,20,20
1,SelfANew,9,10
2,SnausageFest,6,6
3,Pesto_Power,5,5
4,payschlur,5,5
...,...,...,...
225435,zzzz988,0,0
225436,zzzzbear,0,0
225437,zzzzshockedme,0,0
225438,zzzztopportal,0,0


In [11]:
author_df = author_df[author_df.Hurtful != 0] 

In [12]:
author_df = author_df[author_df.Hurtful != 1] 

In [13]:
author_df

,comment_author,Hurtful,Freq hurtful words
0,AutoModerator,20,20
1,SelfANew,9,10
2,SnausageFest,6,6
3,Pesto_Power,5,5
4,payschlur,5,5
...,...,...,...
151,urruke,2,2
152,warped__,2,2
153,wellwithin,2,2
154,zWraith,2,2


In [31]:
def addlist(comment):
    #print(channel)
    channel = reddit.comment(comment)
    return channel.subreddit

listsub = np.vectorize(addlist)

In [22]:
user = reddit.redditor(author_df['comment_author'][90])
k = list(user.comments.new(limit=150))
print(k)

[Comment(id='fobge3i'), Comment(id='fnjof0a'), Comment(id='fmhmaxc'), Comment(id='fmf5q4u'), Comment(id='fmb6cmv'), Comment(id='flopohi'), Comment(id='fkw093d'), Comment(id='fkoz832'), Comment(id='fk2aj24'), Comment(id='fk1ni2c'), Comment(id='fk1hqpn'), Comment(id='fjudbxd'), Comment(id='fjud41v'), Comment(id='fjrj0fu'), Comment(id='fjrhzfs'), Comment(id='fjrh7rq'), Comment(id='fjr9kzg'), Comment(id='fjr96f7'), Comment(id='fjbg5ma'), Comment(id='fj9e3mb'), Comment(id='fj710n3'), Comment(id='fj6devi'), Comment(id='fiku557'), Comment(id='fie5oji'), Comment(id='fidvf9j'), Comment(id='fiba9ra'), Comment(id='fiankej'), Comment(id='fi4voko'), Comment(id='fhek6q3'), Comment(id='fhecju7'), Comment(id='fhe0e7b'), Comment(id='fhdxi0m'), Comment(id='fhcybqe'), Comment(id='fh9eaz6'), Comment(id='fgf7ugy'), Comment(id='fg31z3t'), Comment(id='ffqbgk9'), Comment(id='ffqb3wz'), Comment(id='ff8xinu'), Comment(id='ff3mzww'), Comment(id='fe5a1u4'), Comment(id='fe2dzss'), Comment(id='fe2bwqr'), Comment(id

In [41]:
k[0].subreddit.display_name

AttributeError: 'str' object has no attribute 'k'

In [49]:
import urllib.request
import logging
import prawcore
logger = logging.getLogger(__name__) 

subs = []

for auth in author_df['comment_author']:
    user = reddit.redditor(auth)
    print(user)
    try: 
        k = list(user.comments.new(limit=400))
        subs = np.append(subs, np.array(k))
        #break
    except Exception:
        logger.error('Error retrieving comments for user {}'.format(user))

AutoModerator
SelfANew
SnausageFest
Pesto_Power
payschlur
SuperbWallaby
TheBlindCat
addictedtochips
basherella
WhapXI
AnswerIsItDepends
Ariyanwrynn1989
BewilderedFingers
DiamanteDog
DragonToothGarden
FrugalChef13
JWJulie
MadameTrafficJam
MountainLou
Mystery_Substance
RemtonJDulyak
SecretlyFBI
ShipsOfTheseus8
VacuousWording
amiidala
boudicas_shield
damnyouall2hell
iBeFloe
idontknowanything222
katenax
midnight2196
mollybrains
rockpaperllama
tphatmcgee
AITAMod
ColossusOfChoads
A_Sarcastic_Werecat
Afeazo
AlishaB0214
BagelsAndJewce
BeMyHeroForNow
Beachy5313
BillsInATL
Brightpetals
BurtonIsSexy120
Carnimaniii
Chronicallyoddsgirl
Cookiedoughjunkie
Crossfiyah
DarkRaiiin
DeseretRain
ElizabethHiems
F-r-e-d-F-r-e-d
FatTabby
Friendly_Fire
G4KingKongPun
Garydrgn
GettingCereal
Giengi
GrumpyOldFart74
HerbalCamel
Jasper_J_Jones
JoffreyWaters
Jorojr
JosieAlcott
KatieCashew
KittyLune
MTMD36
MadameHardy
MaryMaryConsigliere
Muninwing
NachoManSandyRavage
Neuroticcuriosity
Nikkoshen
Nougattabekidding
OsamaB

In [50]:
# scale = lambda x: x.subreddit.display_name
# print(scale(subs))
# #subs
#subs[0].subreddit.display_name
arr1 = np.vectorize(lambda x: x.subreddit.display_name)(subs)

In [51]:
import collections 

arr = collections.Counter(arr1)

In [52]:
arr.most_common()

[('AmItheAsshole', 5455),
 ('AskReddit', 1226),
 ('relationship_advice', 421),
 ('relationships', 293),
 ('Random_Acts_Of_Amazon', 237),
 ('Gunners', 195),
 ('childfree', 165),
 ('JUSTNOMIL', 161),
 ('VALORANT', 159),
 ('politics', 153),
 ('soccer', 143),
 ('AmItheButtface', 142),
 ('TrueOffMyChest', 141),
 ('todayilearned', 136),
 ('AnimalCrossing', 122),
 ('Libertarian', 120),
 ('preppers', 119),
 ('thebachelor', 119),
 ('AskAnAmerican', 116),
 ('worldnews', 115),
 ('PurplePillDebate', 115),
 ('Atlanta', 114),
 ('dndnext', 111),
 ('guns', 110),
 ('stopdrinking', 110),
 ('aww', 103),
 ('Cooking', 102),
 ('funny', 89),
 ('news', 84),
 ('Habs', 81),
 ('raisedbynarcissists', 80),
 ('de', 78),
 ('leagueoflegends', 77),
 ('unpopularopinion', 76),
 ('nfl', 76),
 ('JustNoTruth', 74),
 ('personalfinance', 72),
 ('AreTheStraightsOK', 72),
 ('iamatotalpieceofshit', 71),
 ('TigerKing', 71),
 ('fountainpens', 70),
 ('TwoXChromosomes', 66),
 ('insanepeoplefacebook', 64),
 ('femalefashionadvice', 6

In [53]:
print(arr['MensRights'])
print(arr['PussyPass'])
print(arr['badwomensanatomy'])
print(arr['KotakuInAction'])
print(arr['RedPill'])
print(arr['MGTOW'])
print(arr['IncelsWithoutHate'])
print(arr['Braincels'])
print(arr['Truefemcels'])

7
0
43
0
0
0
0
0
0
